In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

Using device: mps


In [3]:
# These are how we are going to transform the images
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [4]:
# load my datasets
train_dataset = datasets.ImageFolder(root='train', transform=transform)
test_dataset = datasets.ImageFolder(root='test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=200, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [5]:
# Create the model to process images
class EmotionClassifier(nn.Module):
    def __init__(self):
        super(EmotionClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 6 * 6, 256)
        self.fc2 = nn.Linear(256, 7)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.dropout(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = self.pool(F.relu(self.conv3(x)))
        x = self.dropout(x)
        x = x.view(-1, 128 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model = EmotionClassifier().to(device)

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [7]:
# Train model
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

Epoch 1/100, Loss: 1.7841
Epoch 2/100, Loss: 1.6987
Epoch 3/100, Loss: 1.6383
Epoch 4/100, Loss: 1.5988
Epoch 5/100, Loss: 1.5666
Epoch 6/100, Loss: 1.5386
Epoch 7/100, Loss: 1.5094
Epoch 8/100, Loss: 1.4847
Epoch 9/100, Loss: 1.4610
Epoch 10/100, Loss: 1.4430
Epoch 11/100, Loss: 1.4216
Epoch 12/100, Loss: 1.4022
Epoch 13/100, Loss: 1.3910
Epoch 14/100, Loss: 1.3701
Epoch 15/100, Loss: 1.3539
Epoch 16/100, Loss: 1.3379
Epoch 17/100, Loss: 1.3251
Epoch 18/100, Loss: 1.3128
Epoch 19/100, Loss: 1.2956
Epoch 20/100, Loss: 1.2835
Epoch 21/100, Loss: 1.2676
Epoch 22/100, Loss: 1.2551
Epoch 23/100, Loss: 1.2443
Epoch 24/100, Loss: 1.2298
Epoch 25/100, Loss: 1.2158
Epoch 26/100, Loss: 1.2067
Epoch 27/100, Loss: 1.1953
Epoch 28/100, Loss: 1.1786
Epoch 29/100, Loss: 1.1709
Epoch 30/100, Loss: 1.1611
Epoch 31/100, Loss: 1.1443
Epoch 32/100, Loss: 1.1327
Epoch 33/100, Loss: 1.1201
Epoch 34/100, Loss: 1.1097
Epoch 35/100, Loss: 1.0969
Epoch 36/100, Loss: 1.0885
Epoch 37/100, Loss: 1.0784
Epoch 38/1

In [8]:
# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total:.2f}%')

Accuracy on test set: 59.40%


In [ ]:
# Save it
torch.save(model.state_dict(), 'emotion_model.pth')